In [1]:
import pandas as pd

In [ ]:
!pip install fbprophet

In [2]:
import fbprophet

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/교량_기준_임시_최종db.csv')

In [5]:
df

,Unnamed: 0,Unnamed: 0.1,bridge_name,address,etc_address,latitude,longitude,brid_height_origin,location_start,wl_station_code,rf_station_code,obs_date,WL,rainfall,bridge_height
0,0,0,신원교,경기도,이천시,37.291282,127.509880,5.4,"[37.291282, 127.50988000000001]",1007645,10074060,2010-02-25 12:00:00,3.0,0.0,510.0
1,1,1,신원교,경기도,이천시,37.291282,127.509880,5.4,"[37.291282, 127.50988000000001]",1007645,10074060,2010-02-25 13:00:00,6.0,0.0,510.0
2,2,2,신원교,경기도,이천시,37.291282,127.509880,5.4,"[37.291282, 127.50988000000001]",1007645,10074060,2010-02-25 14:00:00,10.0,0.0,510.0
3,3,3,신원교,경기도,이천시,37.291282,127.509880,5.4,"[37.291282, 127.50988000000001]",1007645,10074060,2010-02-25 15:00:00,14.0,0.0,510.0
4,4,4,신원교,경기도,이천시,37.291282,127.509880,5.4,"[37.291282, 127.50988000000001]",1007645,10074060,2010-02-25 16:00:00,18.0,0.0,510.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4097827,4097827,4381107,대곡교,서울특별시,강남구,37.468243,127.121565,3.0,"[37.468243, 127.12156499999999]",1018655,10184100,2017-06-26 17:00:00,184.0,0.0,483.0
4097828,4097828,4381108,대곡교,서울특별시,강남구,37.468243,127.121565,3.0,"[37.468243, 127.12156499999999]",1018655,10184100,2017-06-26 18:00:00,184.0,0.0,483.0
4097829,4097829,4381109,대곡교,서울특별시,강남구,37.468243,127.121565,3.0,"[37.468243, 127.12156499999999]",1018655,10184100,2017-06-26 19:00:00,185.0,0.0,483.0
4097830,4097830,4381110,대곡교,서울특별시,강남구,37.468243,127.121565,3.0,"[37.468243, 127.12156499999999]",1018655,10184100,2017-06-26 20:00:00,185.0,0.0,483.0


In [6]:
df['obs_date'] = pd.to_datetime(df['obs_date'])

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4097832 entries, 0 to 4097831
Data columns (total 15 columns):
 #   Column              Dtype         
---  ------              -----         
 0   Unnamed: 0          int64         
 1   Unnamed: 0.1        int64         
 2   bridge_name         object        
 3   address             object        
 4   etc_address         object        
 5   latitude            float64       
 6   longitude           float64       
 7   brid_height_origin  float64       
 8   location_start      object        
 9   wl_station_code     int64         
 10  rf_station_code     int64         
 11  obs_date            datetime64[ns]
 12  WL                  float64       
 13  rainfall            float64       
 14  bridge_height       float64       
dtypes: datetime64[ns](1), float64(6), int64(4), object(4)
memory usage: 469.0+ MB


In [7]:
bridge_list = pd.unique(df['location_start']).tolist()

In [8]:
df_list = [df[df['location_start']==i] for i in bridge_list]

In [9]:
for i in range(len(bridge_list)):
    df_list[i] = df_list[i].drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'bridge_name', 'address', 'etc_address',
                                          'latitude', 'longitude', 'location_start', 'wl_station_code', 'rf_station_code',
                                          'rainfall', 'bridge_height', 'brid_height_origin'])
    df_list[i].rename({'obs_date':'ds', 'WL':'y'}, axis=1, inplace=True)

In [14]:
df_list[0]

,ds,y
0,2010-02-25 12:00:00,3.0
1,2010-02-25 13:00:00,6.0
2,2010-02-25 14:00:00,10.0
3,2010-02-25 15:00:00,14.0
4,2010-02-25 16:00:00,18.0
...,...,...
84484,2020-08-26 21:00:00,12.0
84485,2020-08-26 22:00:00,12.0
84486,2020-08-26 23:00:00,16.0
84487,2020-08-27 00:00:00,17.0


In [10]:
import logging

In [11]:
logging.getLogger('fbprophet').setLevel(logging.WARNING)

In [12]:
from fbprophet import Prophet

In [13]:
m_list = [Prophet() for i in range(len(bridge_list))]

In [14]:
for i in range(len(bridge_list)):
    m_list[i].fit(df_list[i])

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [15]:
future_list = [m_list[i].make_future_dataframe(periods=365, freq='H') for i in range(len(bridge_list))]

In [16]:
future_list[0].tail()

,ds
84849,2020-09-11 02:00:00
84850,2020-09-11 03:00:00
84851,2020-09-11 04:00:00
84852,2020-09-11 05:00:00
84853,2020-09-11 06:00:00


In [17]:
forecast_list = [m_list[i].predict(future_list[i]) for i in range(len(bridge_list))]

In [18]:
forecast_list[0].tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
84849,2020-09-11 02:00:00,-2.172982,-10.619621,42.226631,-2.172982,-2.172982,19.802138,19.802138,19.802138,-0.480277,-0.480277,-0.480277,0.061175,0.061175,0.061175,20.221240,20.221240,20.221240,0.0,0.0,0.0,17.629155
84850,2020-09-11 03:00:00,-2.171062,-10.369018,42.572847,-2.171062,-2.171062,19.816626,19.816626,19.816626,-0.449821,-0.449821,-0.449821,0.065232,0.065232,0.065232,20.201215,20.201215,20.201215,0.0,0.0,0.0,17.645564
84851,2020-09-11 04:00:00,-2.169142,-8.038216,45.328508,-2.169142,-2.169142,19.900779,19.900779,19.900779,-0.348553,-0.348553,-0.348553,0.068303,0.068303,0.068303,20.181029,20.181029,20.181029,0.0,0.0,0.0,17.731637
84852,2020-09-11 05:00:00,-2.167221,-8.627798,44.040294,-2.167221,-2.167221,20.052813,20.052813,20.052813,-0.178047,-0.178047,-0.178047,0.070176,0.070176,0.070176,20.160683,20.160683,20.160683,0.0,0.0,0.0,17.885592
84853,2020-09-11 06:00:00,-2.165301,-8.570579,44.213120,-2.165301,-2.165301,20.234142,20.234142,20.234142,0.023306,0.023306,0.023306,0.070658,0.070658,0.070658,20.140178,20.140178,20.140178,0.0,0.0,0.0,18.068841


In [19]:
for i in range(len(bridge_list)):
    m_list[i].plot(forecast_list[i])

Output hidden; open in https://colab.research.google.com to view.